# Experimento numérico \#4 - Tema Solución de bloques cambiando método de generación de matriz pseudo-aleatoria de sistema.

**Responsable:** León Garay 

## 0. Objetivo

Este documento pretende mostar los resultados obtenidos con la implementación realizada, en el contexto de la implementación de entradas normales con varianza cada vez más alta y matrices de $10^1$, $10^2$.

**Nota:** El reporte general de resultados obtenidos con las implementaciones realizadas en este proyecto se puede consultar en el siguiente [vínculo](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/blob/master/results/Reporte_resultados.ipynb)

## 1. Consideraciones

Al respecto de los experimentos numéricos realizados para consolidar el presente reporte como se ha dicho este pretende analizar los resultados obtenidos con la implementación realizada, restringido a entradas normales con varianza cada vez más alta y matrices de $10^1$, $10^2$.

En este sentido, en particular para cada experimento realizado, se reportan según resulte aplicable:

* los parámetros empleados en la simulaciones, 
* las dimensiones de las matrices y vectores involucrados, así como el prodecimiento pseudo-aleatorio que les dio origen, 
* 1) el tiempo involucrado en correr los experimentos, 
* 2) número de condición de las matrices pseudo-aleatorias, y 
* 3) el error relativo obtenido, para la solución de un sistema lineal de la forma $Ax=b$, es decir el cociente

$$|| Ax - b ||_2 / || b ||_2$$
* 4) La norma de $A$ y de la matriz obtenida al rearmar A tras multiplica las matrices resultantes de la aproximación de la descomposición SVD vía el algoritmo **One-Sided Jacobi**


### 1.1 Consideraciones sobre la infraestructura empleada

**Especificaciones de ambiente común de trabajo**

Para realizar el presente experimento numérico se ha empleado la imagen de docker basada en R del curso MNO 2020 (palmoreck/jupyterlab_r_kernel:1.1.0)

```
docker run --rm -v `pwd`:/datos --name jupyterlab_r_kernel_local -p 8888:8888 -d palmoreck/jupyterlab_r_kernel:1.1.0

```

*Nota:* el comando "-v \`pwd\`:/datos", permite montar el directorio actual en donde el usuario se encuentre situada en terminal como un volumen de la imagen de docker, dentro del directorio "/datos".

## 2. Experimento numérico

**Objetivo**

Modificar las entradas de la matriz y las dimensiones de los bloques para notar que efecto tiene sobre el error relativo y el tiempo de ejecución.

A tal respecto, se destaca que se realizó este experimentos buscando probar el comportamiento correcto de la función y medir los tiempo de ejecución.

**Cargamos codigo desarrollado previamente**

* **utils.R:** contiene las funciones auxiliares desarrolladas para el proyecto
* **00-load.R:** la implementación del método de eliminación por bloques, empleando la aproximación de la descomposición SVD vía el algoritmo **One-Sided Jacobi**.

In [1]:
## Instalamos paquetes
rm(list = ls())

paquetes <- c('matrixcalc')

instalar <- function(paquete) {
  if (!require(paquete,character.only = TRUE, quietly = TRUE, warn.conflicts = FALSE)) {
    install.packages(as.character(paquete), dependecies = TRUE, repos = "http://cran.us.r-project.org")
    library(paquete, character.only = TRUE, quietly = TRUE, warn.conflicts = FALSE)
  }
}


lapply(paquetes, instalar)

[[1]]
NULL

In [2]:
## Cargamos paquetes necesarios
library("matrixcalc")
library("matlib")
#source("metadata.R")
source("utils.R")
source("00-load.R")


Attaching package: 'matlib'

The following object is masked from 'package:matrixcalc':

    vec



In [3]:
install.packages("tictoc",lib="/usr/local/lib/R/site-library/",
                repos="https://cran.itam.mx/",verbose=TRUE)

Warning message in install.packages("tictoc", lib = "/usr/local/lib/R/site-library/", :
"'lib = "/usr/local/lib/R/site-library/"' is not writable"

ERROR: Error in install.packages("tictoc", lib = "/usr/local/lib/R/site-library/", : unable to install packages


In [4]:
library('tictoc')

### 2.1 Experimento 4.1 

Este experimento se basa en los siguientes parámetros [Pendiente: por favor ajustar tabla según experimento]

| # |         Parámetros         | Valor/Rango de valores | Comentarios |
|:-:|:--------------------------:|:----------------------:|:-----------:|
| 1 |     Dimensiones de $A$     |      10x10             |             |
| 2 |        Dimensión $b$       |        10x1            |             |
| 3 |         Tolerancia         |      10**-8            |             |
| 4 |          Maxsweep          |        20              |             |
| 5 |   Metodo pseudo-aleatorio  |     números reales     |        Obtenido con método rnorm(n)    |
| 6 | Numero de condición de $A$ |      9.7491            |     Obtenido con función $||A||*||A^{-1}||$        |

**Codigo:** Dada una matriz A esta se subdivide en bloques para resolver el sistema $Ax=b$.

In [5]:
set.seed(231)
n= 10**1

A = matrix(rnorm(n**2), ncol=n)
b = matrix(rnorm(n), ncol=1)
TOL = 10**-8
maxsweep <- 20
tic()
z <- eliminacion_bloques(A,b,n/2,TOL,maxsweep)
toc()

0.71 sec elapsed


In [6]:
# Error relativo
norm(A%*%z-b,"2")/norm(b,"2")

[1] 1.810544

In [7]:
# Error factorización
svd_aprox <- svd_jacobi_aprox(A, TOL, maxsweep)
norm(A-svd_aprox$U%*%svd_aprox$S%*%t(svd_aprox$V),'2')

[1] 8.840715e-15

In [8]:
# Condición de A
norm(A, "2")*norm(inv(A),"2")

[1] 9.749164

**Resultados**

Los resultados obtenidos se resumen a través de la siguiente tabla 

| # |                Parámetros                | Valor/Rango de valores |            Comentarios            |
|:-:|:----------------------------------------:|:----------------------:|:---------------------------------:|
| 1 |            Tiempo de ejecución           |           0.71 s        |                                   |
| 2 |      Error relativo $||Ax-b||/||b||$     |         1.8105         |                                   |
| 3 | Error factorizacion SVD $||Ax-b||/||b||$ |        8.84071e-15          | norma $A-U\cdot \Sigma \cdot V^T$ |


### 2.2  Experimento 4.2

Este experimento se basa en los siguientes parámetros

| # |         Parámetros         | Valor/Rango de valores | Comentarios |
|:-:|:--------------------------:|:----------------------:|:-----------:|
| 1 |     Dimensiones de $A$     |      100x100             |             |
| 2 |        Dimensión $b$       |        100x1            |             |
| 3 |         Tolerancia         |      10**-8            |             |
| 4 |          Maxsweep          |        20              |             |
| 5 |   Metodo pseudo-aleatorio  |     números reales     |        Obtenido con método rnorm(n)    |
| 6 | Numero de condición de $A$ |      109.1753            |     Obtenido con función $||A||*||A^{-1}||$        |

**Codigo:** Dada una matriz A esta se subdivide en bloques para resolver el sistema $Ax=b$.

In [9]:
set.seed(231)
n= 10**2

A = matrix(rnorm(n**2,0,10), ncol=n)
b = matrix(rnorm(n,0,10), ncol=1)
TOL = 10**-8
maxsweep <- 20
tic()
z <- eliminacion_bloques(A,b,n/2,TOL,maxsweep)
toc()

371.67 sec elapsed


In [10]:
# Error relativo
norm(A%*%z-b,"2")/norm(b,"2")

[1] 0.8240097

In [11]:
# Error factorización
svd_aprox <- svd_jacobi_aprox(A, TOL, maxsweep)
norm(A-svd_aprox$U%*%svd_aprox$S%*%t(svd_aprox$V),'2')

[1] 2.676068e-12

In [12]:
# Condición de A
norm(A, "2")*norm(inv(A),"2")

[1] 109.1753

**Resultados**

Los resultados obtenidos se resumen a través de la siguiente tabla 

| # |                Parámetros                | Valor/Rango de valores |            Comentarios            |
|:-:|:----------------------------------------:|:----------------------:|:---------------------------------:|
| 1 |            Tiempo de ejecución           |     371.67 s           |                                   |
| 2 |      Error relativo $||Ax-b||/||b||$     |      0.82400           |                                   |
| 3 | Error factorizacion SVD $||Ax-b||/||b||$ |     2.676068e-12          | norma $A-U\cdot \Sigma \cdot V^T$ |

**Resumen de resultados obtenidos en experimentos**

Los resultados obtenidos, se compararan a través de la siguiente:

| # | Experimento | Tiempo de desempeño | Error relativo | Error factorización | Comentarios |
|:-:|:-----------:|:-------------------:|----------------|----------------|:-----------:|
| 1 |     4.1     |       0.71 s        |     1.8105     |  8.84071e-15  |             |
| 2 |     4.2     |      371.67 s            |    0.82400   |   2.676068e-12   |             |


## 3. Principales hallazgos

Al respecto, se destacan los siguientes hallazgos del experimento:

* Hallazgo 1: El algoritmo fue consistente al aumentar la dimensión de la matriz y la varianza de las entradas.
* Hallazgo 2: El tiempo de ejecución aumentó considerablemente al aumentar la dimensión de la matriz.
